Импортируем необходимые библиотеки

In [49]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from skopt.space import Real, Integer
from skopt import BayesSearchCV
from sklearn.metrics import r2_score, accuracy_score

Загрузим датасет и посмотрим на признаки

In [4]:
df = pd.read_csv('Sleep_health_and_lifestyle_dataset.csv')
df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           155 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB


In [6]:
df['Sleep Disorder'].value_counts()

Sleep Disorder
Sleep Apnea    78
Insomnia       77
Name: count, dtype: int64

In [7]:
df['Sleep Disorder'] = df['Sleep Disorder'].fillna('No information')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           374 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB


In [9]:
df.isna().sum()

Person ID                  0
Gender                     0
Age                        0
Occupation                 0
Sleep Duration             0
Quality of Sleep           0
Physical Activity Level    0
Stress Level               0
BMI Category               0
Blood Pressure             0
Heart Rate                 0
Daily Steps                0
Sleep Disorder             0
dtype: int64

In [10]:
df.duplicated().sum()

0

Изучим статистические данные

In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Person ID,374.0,187.500000,108.108742,1.0,94.25,187.5,280.75,374.0
Age,374.0,42.184492,8.673133,27.0,35.25,43.0,50.00,59.0
Sleep Duration,374.0,7.132086,0.795657,5.8,6.40,7.2,7.80,8.5
Quality of Sleep,374.0,7.312834,1.196956,4.0,6.00,7.0,8.00,9.0
Physical Activity Level,374.0,59.171123,20.830804,30.0,45.00,60.0,75.00,90.0
Stress Level,374.0,5.385027,1.774526,3.0,4.00,5.0,7.00,8.0
Heart Rate,374.0,70.165775,4.135676,65.0,68.00,70.0,72.00,86.0
Daily Steps,374.0,6816.844920,1617.915679,3000.0,5600.00,7000.0,8000.00,10000.0


In [12]:
df.describe(include='object').T

,count,unique,top,freq
Gender,374,2,Male,189
Occupation,374,11,Nurse,73
BMI Category,374,4,Normal,195
Blood Pressure,374,25,130/85,99
Sleep Disorder,374,3,No information,219


In [13]:
df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,No information
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,No information
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,No information
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


Создадим нолвые признаки из уже имеющихся

In [14]:
df['Blood Pressure'].value_counts()

Blood Pressure
130/85    99
140/95    65
125/80    65
120/80    45
115/75    32
135/90    27
140/90     4
125/82     4
132/87     3
128/85     3
126/83     2
115/78     2
139/91     2
142/92     2
119/77     2
135/88     2
129/84     2
128/84     2
131/86     2
117/76     2
130/86     2
118/75     2
121/79     1
122/80     1
118/76     1
Name: count, dtype: int64

In [15]:
# Разделения значения давления и преобразование в числовой формат
df[['Systilic BP', 'Diastolic BP']] = df['Blood Pressure'].str.split('/', expand=True).astype(int)

# Разбиение на категории по систолическому давлению
df['Systolic Category'] = pd.cut(df['Systilic BP'], bins=[0, 119, 129, 139, 159, 180], labels=[0, 1, 2, 3, 4], right=False)

# Разбиение на категории по диаистолическому давлению
df['Diastolic Category'] = pd.cut(df['Diastolic BP'], bins=[0, 79, 89, 99, 109, 120], labels=[0, 1, 2, 3, 4], right=False)

print(df[['Systilic BP', 'Systolic Category', 'Diastolic BP', 'Diastolic Category']].head())

   Systilic BP Systolic Category  Diastolic BP Diastolic Category
0          126                 1            83                  1
1          125                 1            80                  1
2          125                 1            80                  1
3          140                 3            90                  2
4          140                 3            90                  2


In [16]:
df['Gender'].value_counts()

Gender
Male      189
Female    185
Name: count, dtype: int64

In [17]:
df['Gender'] = df['Gender'].replace({'Female': 0, 'Male': 1})

In [18]:
df = pd.get_dummies(df, columns=['Sleep Disorder'], drop_first=True)

In [19]:
df['Occupation'].value_counts()

Occupation
Nurse                   73
Doctor                  71
Engineer                63
Lawyer                  47
Teacher                 40
Accountant              37
Salesperson             32
Software Engineer        4
Scientist                4
Sales Representative     2
Manager                  1
Name: count, dtype: int64

In [20]:
frequency = df['Occupation'].value_counts(normalize=True)
df['Occupation'] = df['Occupation'].map(frequency)
df['Occupation'].value_counts()

Occupation
0.195187    73
0.189840    71
0.168449    63
0.125668    47
0.106952    40
0.098930    37
0.085561    32
0.010695     8
0.005348     2
0.002674     1
Name: count, dtype: int64

In [21]:
df['BMI Category'].value_counts()

BMI Category
Normal           195
Overweight       148
Normal Weight     21
Obese             10
Name: count, dtype: int64

In [23]:
df['BMI Category'] = df['BMI Category'].replace({'Normal Weight': 'Normal'})
df['BMI Category Coded'] = pd.Categorical(df['BMI Category'], categories=['Normal', 'Overweight', 'Obese'], ordered=True)
df['BMI Category Coded'] = df['BMI Category Coded'].cat.codes

print(df['BMI Category Coded'].value_counts())

BMI Category Coded
0    216
1    148
2     10
Name: count, dtype: int64


In [24]:
df.drop(['Blood Pressure', 'BMI Category', 'Person ID'], axis=1, inplace=True)

In [25]:
df.head()

,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Heart Rate,Daily Steps,Systilic BP,Diastolic BP,Systolic Category,Diastolic Category,Sleep Disorder_No information,Sleep Disorder_Sleep Apnea,BMI Category Coded
0,1,27,0.010695,6.1,6,42,6,77,4200,126,83,1,1,True,False,1
1,1,28,0.189840,6.2,6,60,8,75,10000,125,80,1,1,True,False,0
2,1,28,0.189840,6.2,6,60,8,75,10000,125,80,1,1,True,False,0
3,1,28,0.005348,5.9,4,30,8,85,3000,140,90,3,2,False,True,2
4,1,28,0.005348,5.9,4,30,8,85,3000,140,90,3,2,False,True,2


Используем полиномиальные признаки для числовых переменных

In [26]:
features = df[['Age', 'Daily Steps', 'Heart Rate']]

poly = PolynomialFeatures(degree=2, include_bias=False)                        
poly_features = poly.fit_transform(features)
poly_features_df = pd.DataFrame(poly_features, columns=poly.get_feature_names_out(features.columns))

df = pd.concat([df, poly_features_df], axis=1)

df.head()                              

,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Heart Rate,Daily Steps,Systilic BP,...,BMI Category Coded,Age,Daily Steps,Heart Rate,Age^2,Age Daily Steps,Age Heart Rate,Daily Steps^2,Daily Steps Heart Rate,Heart Rate^2
0,1,27,0.010695,6.1,6,42,6,77,4200,126,...,1,27.0,4200.0,77.0,729.0,113400.0,2079.0,17640000.0,323400.0,5929.0
1,1,28,0.189840,6.2,6,60,8,75,10000,125,...,0,28.0,10000.0,75.0,784.0,280000.0,2100.0,100000000.0,750000.0,5625.0
2,1,28,0.189840,6.2,6,60,8,75,10000,125,...,0,28.0,10000.0,75.0,784.0,280000.0,2100.0,100000000.0,750000.0,5625.0
3,1,28,0.005348,5.9,4,30,8,85,3000,140,...,2,28.0,3000.0,85.0,784.0,84000.0,2380.0,9000000.0,255000.0,7225.0
4,1,28,0.005348,5.9,4,30,8,85,3000,140,...,2,28.0,3000.0,85.0,784.0,84000.0,2380.0,9000000.0,255000.0,7225.0


Применим несколько методов отбора признаков

In [27]:
X = df.drop('Quality of Sleep', axis=1)
y = df['Quality of Sleep']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
model1 = LogisticRegression()

model1.fit(X_train, y_train)
prediction = model1.predict(X_test)

print(f'R^2 на тренировочных данных {r2_score(y_train, model1.predict(X_train))}')
print(f'R^2 на тестовых данных {r2_score(y_test, model1.predict(X_test))}')

R^2 на тренировочных данных 0.08930603869768039
R^2 на тестовых данных 0.3194673580014141


In [30]:
coefficients = model1.coef_
feature_names = X_train.columns

feature_importance = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients[0]})
feature_importance = feature_importance.sort_values(by='Coefficient', ascending=False)
feature_importance

,Feature,Coefficient
22,Daily Steps Heart Rate,4.979667e-06
23,Heart Rate^2,1.629566e-07
16,Daily Steps,2.774092e-08
7,Daily Steps,2.774092e-08
20,Age Heart Rate,1.514477e-08
8,Systilic BP,1.906687e-09
6,Heart Rate,1.551081e-09
17,Heart Rate,1.551081e-09
9,Diastolic BP,1.251687e-09
5,Stress Level,2.305948e-10


In [31]:
from sklearn.feature_selection import RFE
# Создаем RFE модель и выбираем 10 признака
rfe = RFE(estimator=model1, n_features_to_select=10)
rfe = rfe.fit(X, y)

# Показываем какие признаки выбраны
selected_features = pd.DataFrame({
    'Feature': X.columns,  # Здесь X.columns должен содержать названия столбцов, использованных для X
    'Ranking': rfe.ranking_
})
print(selected_features.sort_values(by='Ranking', ascending=False))

                          Feature  Ranking
2                      Occupation       15
13     Sleep Disorder_Sleep Apnea       14
0                          Gender       13
12  Sleep Disorder_No information       12
10              Systolic Category       11
14             BMI Category Coded       10
11             Diastolic Category        9
3                  Sleep Duration        8
5                    Stress Level        7
1                             Age        6
15                            Age        5
9                    Diastolic BP        4
6                      Heart Rate        3
17                     Heart Rate        2
19                Age Daily Steps        1
22         Daily Steps Heart Rate        1
21                  Daily Steps^2        1
20                 Age Heart Rate        1
4         Physical Activity Level        1
18                          Age^2        1
16                    Daily Steps        1
8                     Systilic BP        1
7          

In [33]:
top_features = selected_features.sort_values(by='Ranking', ascending=False)[:11]
X_train_selected = X_train[top_features.Feature]
X_test_selected = X_test[top_features.Feature]

model2 = LogisticRegression(random_state=42)
model2.fit(X_train_selected, y_train)
print(f'R^2 на тренировочных данных {r2_score(y_train, model2.predict(X_train_selected))}')
print(f'R^2 на тестовых данных {r2_score(y_test, model2.predict(X_test_selected))}')

R^2 на тренировочных данных 0.9352927974864141
R^2 на тестовых данных 0.9204572236625029


Видно, что отобрав наиболее 10 важных признаков, созданная модель лучше отрабатывает. Коэффициент R^2 выше как на тренировочных, так и тестовых данных чем у модели, использующая все признаки в датафрейме.

Используем GridSearchCV и BayesSearchCV для настройки параметров модели регрессии. Оценим, как изменение гиперпараметров влияет на качество предсказаний.

In [35]:
param_grid = {'C': [0.01, 0.1, 1, 10, 100]}

In [37]:
model_l1 = LogisticRegression(penalty='l1', solver='liblinear')

In [38]:
grid_l1 = GridSearchCV(model_l1, param_grid, cv=5)

In [39]:
grid_l1.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=LogisticRegression(penalty='l1', solver='liblinear'),
             param_grid={'C': [0.01, 0.1, 1, 10, 100]})

In [40]:
best_model_l1 = grid_l1.best_estimator_

In [41]:
# Оцениваем качество моделей на тренировочных и тестовых данных
print('Качество модели L1:')
print(f'Оптимальное значение параметра регуляризации: {grid_l1.best_params_}')
print(f'Качество модели на тренировочных данных: {accuracy_score(y_train, best_model_l1.predict(X_train))}')
print(f'Качество модели на тестовых данных: {accuracy_score(y_test, best_model_l1.predict(X_test))}')

Качество модели L1:
Оптимальное значение параметра регуляризации: {'C': 10}
Качество модели на тренировочных данных: 0.9899665551839465
Качество модели на тестовых данных: 0.96


In [43]:
search_space = {
    "C": Real(1e-3, 10, prior="log-uniform"),
    "penalty": ["l1", "l2"],
    "solver": ["liblinear", "saga"],
}

In [44]:
model = BayesSearchCV(
    estimator=LogisticRegression(),
    search_spaces=search_space,
    n_iter=50,  # Количество итераций поиска
    cv=5,  # Количество кросс-валидаций
    scoring="accuracy",
    random_state=42,
)

In [45]:
model.fit(X_train, y_train)

BayesSearchCV(cv=5, estimator=LogisticRegression(), random_state=42,
              scoring='accuracy',
              search_spaces={'C': Real(low=0.001, high=10, prior='log-uniform', transform='normalize'),
                             'penalty': ['l1', 'l2'],
                             'solver': ['liblinear', 'saga']})

In [46]:
# Вывод лучших гиперпараметров
print("Лучшие гиперпараметры:", model.best_params_)

Лучшие гиперпараметры: OrderedDict([('C', 9.928010365023384), ('penalty', 'l1'), ('solver', 'liblinear')])


In [47]:
# Прогнозирование на тестовой выборке
y_pred = model.predict(X_test)

In [48]:
# Вычисление точности
accuracy = accuracy_score(y_test, y_pred)
print("Точность на тестовой выборке:", accuracy)

Точность на тестовой выборке: 0.96


Исходя из представленных результатов, оба метода оптимизации гиперпараметров (Grid Search и Bayesian Optimization) показали одинаково высокую эффективность accuracy_score, равной 0.96. Это говорит о том, что обе настройки смогли подобрать параметры, которые эффективно работают для моделирования с имеющимися данными. То, что оба подхода выдали высокие значения accuracy_score, предполагает, что модель хорошо аппроксимирует с нашими данными.